In [ ]:
import requests
import pandas as pd
import time
from itertools import product

# Define your parameters
instrument = "Index Futures"  # Index futures
symbols = ["NIFTY", "BANKNIFTY", "FINNIFTY", "MIDCPNIFTY", "CNXIT", "NIFTYINFRA", "NIFTYFMCG", "NIFTYMIDCAP150", 
           "NIFTYSMALLCAP250", "NIFTYPSE", "NIFTYPVTBANK", "NIFTYAUTO", "NIFTYMEDIA", "NIFTYPHARMA", 
           "NIFTYREALTY", "NIFTYENERGY", "NIFTY50VALUE20", "NIFTYALPHA50", "NIFTYCPSE", "NIFTY100", "NIFTY200"]
years = ["2024", "2023"]
expiries = ["29-Jun-2023", "27-Jul-2023"]  # Example expiry dates with proper formatting

# Function to fetch data
def fetch_nse_data(symbol, year, expiry):
    # Construct the URL (adjust based on the actual NSE API)
    url = "https://www.nseindia.com/report-detail/fo_eq_security"
    
    # Headers to mimic a request from a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
    }
    
    # Use a session to handle cookies and other settings
    session = requests.Session()
    session.headers.update(headers)
    
    # Make an initial request to get cookies
    session.get("https://www.nseindia.com")
    
    # Make the GET request with parameters
    params = {"symbol": symbol}
    response = session.get(url, headers=headers, params=params)

    if response.status_code == 200:
        try:
            # Parse the JSON data
            data = response.json()
            
            # Convert to DataFrame (adjust based on the actual JSON structure)
            records = data.get('records', {}).get('data', [])
            df = pd.DataFrame(records)
            
            # Construct a filename
            filename = f'nse_data_{instrument}_{symbol}_{year}_{expiry}.csv'
            
            # Save to CSV
            df.to_csv(filename, index=False)
            
            print(f"Data fetched and saved successfully for {instrument} {symbol} {year} {expiry}!")
        except Exception as e:
            print(f"Error processing data for {instrument} {symbol} {year} {expiry}: {e}")
    else:
        print(f"Failed to fetch data for {instrument} {symbol} {year} {expiry}. Status code: {response.status_code}")

# Function to fetch all data
def fetch_all_nse_data():
    # Iterate through all combinations of parameters
    for symbol, year, expiry in product(symbols, years, expiries):
        fetch_nse_data(symbol, year, expiry)

# Function to run the task daily
def run_daily_at(hour, minute, task):
    while True:
        # Get the current time
        now = time.localtime()
        current_time = (now.tm_hour, now.tm_min)
        
        # Check if the current time matches the scheduled time
        if current_time == (hour, minute):
            task()
            # Wait for a minute to avoid running the task multiple times within the same minute
            time.sleep(60)
        else:
            # Sleep for a short time to avoid busy waiting
            time.sleep(10)

# Schedule the fetch_all_nse_data function to run daily at 09:00
run_daily_at(9, 0, fetch_all_nse_data)
